In [1]:
!pip install llama-index-vector-stores-faiss
!pip install llama-index-readers-file


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install llama-index PyMuPDF


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install llama-index-llms-openai-like


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from llama_index.core import SimpleDirectoryReader

loader = SimpleDirectoryReader(input_dir="../data", required_exts=".txt")
documents = loader.load_data()
#print(documents[:2])

In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
import os
from dotenv import load_dotenv
from llama_index.core.embeddings.utils import EmbedType, resolve_embed_model
from llama_index.core.node_parser import SentenceSplitter

load_dotenv()

# Set up the OpenAI LLM
Settings.llm = OpenAILike(
  model='tentris',
  api_key=os.getenv("TENTRIS_API_KEY"),
  api_base=os.getenv("TENTRIS_BASE_URL_CHAT"),

)

Settings.embed_model = OpenAIEmbedding(
    api_base=os.getenv("TENTRIS_BASE_URL_EMBEDDINGS"),
    api_key=os.getenv("TENTRIS_API_KEY"),
    model_name='tentris',
)

# Create index

splitter = SentenceSplitter(chunk_size=512)
index = VectorStoreIndex.from_documents(
    documents, 
    show_progress=True)

c:\Users\KUNJAN SHAH\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 12/12 [00:00<00:00, 13.00it/s]


In [6]:
chat_engine = index.as_chat_engine(chat_mode="condense_plus_context")

In [7]:
str(chat_engine.chat("Who is caglar demir?"))


' Caglar Demir is a Senior Researcher at DICE Research, located'

In [ ]:
from giskard.rag import KnowledgeBase, generate_testset, QATestset
import pandas as pd
import giskard


giskard.llm.set_embedding_model(
        "openai/tentris",  
        api_key=os.getenv("TENTRIS_API_KEY"),
        api_base=os.getenv("TENTRIS_BASE_URL_EMBEDDINGS")
    )

giskard.llm.set_llm_model(
    "openai/tentris",  
    api_key=os.getenv("TENTRIS_API_KEY"),
    api_base=os.getenv("TENTRIS_BASE_URL_CHAT")
)

text_nodes = splitter(documents)
#print(text_nodes[:])
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes])
knowledge_base = KnowledgeBase(knowledge_base_df)

[TextNode(id_='bd2600e2-997d-49c0-afa0-eb9655822a75', embedding=None, metadata={'file_path': 'd:\\Job\\7.Dice_WHB\\3.Dice_tasks\\DICE_group-GraphRAG\\Langchain-RAG\\benchmark_gen\\..\\data\\speech.txt', 'file_name': 'speech.txt', 'file_type': 'text/plain', 'file_size': 32212, 'creation_date': '2025-02-15', 'last_modified_date': '2025-02-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='79f77d23-1c0f-4ac5-b5af-d926fca2351c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'd:\\Job\\7.Dice_WHB\\3.Dice_tasks\\DICE_group-GraphRAG\\Langchain-RAG\\benchmark_gen\\..\\data\\speech.txt', 'file_name': 'speech.txt', 'file_type': 'text/plain', 'file_size': 32212, 'creation_date': '2025-02-15', 'last_mo

In [14]:
testset = generate_testset(knowledge_base, 
                           num_questions=2)

2025-02-17 00:29:08,551 pid:19200 MainThread giskard.rag  INFO     Finding topics in the knowledge base.

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



c:\Users\KUNJAN SHAH\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


2025-02-17 00:29:11,821 pid:19200 MainThread giskard.rag  INFO     Found 3 topics in the knowledge base.


Generating questions: 100%|██████████| 2/2 [00:01<00:00,  1.13it/s]


In [15]:
testset.save("testset.jsonl")
testset = QATestset.load("testset.jsonl")

In [16]:
from giskard.rag import evaluate, RAGReport, AgentAnswer
from giskard.rag.metrics.ragas_metrics import ragas_context_recall, ragas_context_precision

In [17]:
def answer_fn(question, history=None):

    answer = chat_engine.chat(question, chat_history=[])

    return AgentAnswer(
        message=answer.response,
        documents=[source.content for source in answer.sources]
    )

report = evaluate(answer_fn, 
                testset=testset, 
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_context_precision])
report.save("test_report")

RagasMetric evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

2025-02-17 00:29:28,395 pid:19200 MainThread ragas.llms.base WARNING  is_finished not implemented for RagasLLMWrapper. Will default to True.


RagasMetric evaluation:  50%|█████     | 1/2 [00:01<00:01,  1.42s/it]

2025-02-17 00:29:29,327 pid:19200 MainThread ragas.llms.base WARNING  is_finished not implemented for RagasLLMWrapper. Will default to True.


RagasMetric evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

2025-02-17 00:29:30,225 pid:19200 MainThread ragas.llms.base WARNING  is_finished not implemented for RagasLLMWrapper. Will default to True.


RagasMetric evaluation:  50%|█████     | 1/2 [00:00<00:00,  1.13it/s]

2025-02-17 00:29:30,988 pid:19200 MainThread ragas.llms.base WARNING  is_finished not implemented for RagasLLMWrapper. Will default to True.


RagasMetric evaluation: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s]


KeyError: 'correctness'

In [ ]:
report.save("test_report")